In [1]:
import pyomo.environ as pyo
import numpy as np
import json
import pandas as pd
from omlt import OmltBlock, OffsetScaling
from omlt.neuralnet import ReluComplementarityFormulation, ReluPartitionFormulation
from omlt.io.keras import keras_reader
from tensorflow import keras
from omlt.io.onnx import write_onnx_model_with_bounds, load_onnx_neural_network_with_bounds
import os
from sklearn.model_selection import train_test_split
from datetime import datetime
import contextlib

## creating scaler and input bounds for NN definition

In [3]:
#JSON files that contain simulation outputs
simulation_files = ['simulation_DIST_0-862 7.1.24.json','simulation_DIST_826-2461 7.2.24.json','simulation_DIST_2462-6462 7.2.24.json']
values = [] #feed_a, feed_b, feed_c, dist_a, dist_b, dist_c, bott_a, bott_b, bott_c, cond_duty, reb_duty, # trays, feed tray,dist_rate,reflux_ratio
components = ['N-BUTANE','N-PENTAN','N-HEXANE']
block_vals = ["Investment Cost","Operating Cost Reboiler","Operating Cost Condenser",'#-TRAYS','FEED_TRAY','DIST_RATE','REFLUX_RATIO','COND-DUTY']

for f in simulation_files:
    with open(f,"r") as file:
        jsonData = json.load(file)
        
        for i in range(1,len(jsonData)+1):
            values_temp = [] 
            for j in components:
                #print(jsonData[str(i)]['FEED']['MOLEFLOW'][j])
                values_temp.append(jsonData[str(i)]['FEED']['MOLEFLOW'][j])
            for j in components:
                values_temp.append(jsonData[str(i)]['DIST']['MOLEFLOW'][j])
            for j in components:
                values_temp.append(jsonData[str(i)]['BOTT']['MOLEFLOW'][j])
            for j in block_vals:
                values_temp.append(jsonData[str(i)]['B1'][j])
            values.append(values_temp)
            
df = pd.DataFrame(values, columns=['FEED_BUT','FEED_PEN','FEED_HEX','DIST_BUT','DIST_PEN','DIST_HEX','BOTT_BUT',
                                             'BOTT_PEN','BOTT_HEX',"Investment Cost","Operating Cost Reboiler","Operating Cost Condenser",'#-TRAYS','FEED_TRAY','DIST_RATE','REF_RAT','COND_DUTY'])


df = df[np.array(df['FEED_TRAY']) / np.array(df['#-TRAYS']) <=.5]
df = df[~df['COND_DUTY'].isin(['Returned None'])] #getting rid of simulations that had errors
df = df[~df['Investment Cost'].isin(['Returned None'])] #getting rid of simulations that had errors

df_err = df[df['COND_DUTY'] >=0] #getting rid of simulations that have unphysical reboiler duty values
df = df[df['COND_DUTY'] < 0]

df['COND_DUTY'] = np.array(df['COND_DUTY'])/ 1e3 #scaling units to be kilocalories / s
df['Operating Cost Reboiler'] = 1.07 * np.array(df['Operating Cost Reboiler']) / 1e4 #scaling units to be 10,000$/year 1.07 scaler is for converting euro to usd
df['Investment Cost'] = 1.07 * np.array(df['Investment Cost']) / 1e4
df['Operating Cost Condenser'] = 1.07 * np.array(df['Operating Cost Condenser']) / 1e4
df['Operating Cost'] = np.array(df['Operating Cost Condenser']) + np.array(df['Operating Cost Reboiler'])
df['Annual Cost'] = np.array(df['Operating Cost'] + df['Investment Cost'])

inputs = ['FEED_BUT','FEED_PEN','FEED_HEX','#-TRAYS','FEED_TRAY','DIST_RATE','REF_RAT']
outputs = ['DIST_BUT','DIST_PEN','DIST_HEX',"Investment Cost",'Operating Cost']
X_ab = df[inputs]
y_ab = df[outputs]
x_offset_ab, x_factor_ab = X_ab.mean().to_dict(), X_ab.std().to_dict()
y_offset_ab, y_factor_ab = y_ab.mean().to_dict(), y_ab.std().to_dict()

Xs_ab = (X_ab - X_ab.mean()).divide(X_ab.std())
ys_ab = (y_ab - y_ab.mean()).divide(y_ab.std())
scaled_lb_ab = Xs_ab.min()[inputs].values
scaled_ub_ab = Xs_ab.max()[inputs].values
scaler_ab = OffsetScaling(
        offset_inputs={i: x_offset_ab[inputs[i]] for i in range(len(inputs))},
        factor_inputs={i: x_factor_ab[inputs[i]] for i in range(len(inputs))},
        offset_outputs={i: y_offset_ab[outputs[i]] for i in range(len(outputs))},
        factor_outputs={i: y_factor_ab[outputs[i]] for i in range(len(outputs))}
    )
scaler_bc = scaler_ab
input_bounds_ab = {i: (scaled_lb_ab[i], scaled_ub_ab[i]) for i in range(len(inputs))}
input_bounds_bc = input_bounds_ab
offset_outputs={i: y_offset_ab[outputs[i]] for i in range(len(outputs))}
print(offset_outputs)

{0: 53.80698449454978, 1: 35.98511648384938, 2: 14.534259008803412, 3: 92.6995069819045, 4: 72.18255176025893}


## Formulation

In [6]:
dist_rate_range = (.25,.75) #25% of column input stream to 75%
feed_rate = 1000
mol_A = .33 #kmol/hr
mol_B = .33 #kmol/hr
mol_C = .34 #kmol/hr
model = pyo.ConcreteModel()

#sets
model.COMP = pyo.Set(initialize=['A','B','C']) #A is n-butene, B is n-pentene, C is n-hexane
model.COLUMNS = pyo.Set(initialize=['A|BC','AB|C'])
model.STREAMS = pyo.Set(initialize=['F0','F1','F2','F3','F4','F5','F6','F7','F8','F9','F10','F11',
                               'F12','F13','F14','F15','F16','F17','F18','F19','F20','F21',
                               'F22','F23']) #when referencing superstructure picture, subtract all streams by 1 so feed is 0 now and the last stream is 23
model.PROD = pyo.Set(initialize=['P1','P2']) #products

model.si = pyo.RangeSet(0,6) #index set to access splitters 2-8 in constraints
model.ci = pyo.RangeSet(1,2) #index set to access columns 1-2
model.fi = pyo.RangeSet(1,3) #index set to access streams connected to feed
model.mci = pyo.RangeSet(0,1) #index set to access mixers 1 and 2
model.mcp = pyo.RangeSet(0,1) #index set to access mixers 3 and 4
#model.str_i = pyo.Set(initialize=[i for i in range(1, 23) if i not in [4,9]])
model.str_i = pyo.RangeSet(1,23)
inlet_split = [6,10,5,8,12,11,3] #indexes of streams that go into splitter to be called by model.f[i]
outlet_split = [[1,2,3],[8,7],[13,12],[14,15],[16,17],[18,19],[20,21],[22,23]] #indexes of streams that go out of the splitter. First index corresponds to feed stream, not any of the streams in inlet_split list.
inlet_column = [4,9] #inlet streams going into columns
outlet_column = [[5,6],[10,11]]
inlet_col_mix = [[1,13],[2,7]] #streams going into mixers before the column
outlet_col_mix = [4,9] #streams going out of mixers before column
inlet_prod_mix = [[14,16,18,20,22],[15,17,19,21,23]] #streams going out of mixers 3 and 4 going into products 1 and 2
#data
prod_comp = {
    ('P1','A'):200, ('P1','B'):165,('P1','C'):140,
    ('P2','A'):130, ('P2','B'):165,('P2','C'):200
}

#Parameters
model.prod_comp = pyo.Param(model.PROD,model.COMP,initialize=prod_comp)

#variables
model.fa = pyo.VarList()
for i in range(0,23):
    model.fa.add()
for i in range(1, len(model.fa)+1):
    if i == 4 or i == 9:
        model.fa[i].setlb(0)
        model.fa[i].setub(1000) #can raise upperbound because of recycle stream
    else:
        model.fa[i].setlb(0)
        model.fa[i].setub(1000)
model.fb = pyo.VarList()
for i in range(0,23):
    model.fb.add()
for i in range(1, len(model.fb)+1):
    if i == 4 or i == 9:
        model.fb[i].setlb(0)
        model.fb[i].setub(1000) #can raise upperbound because of recycle stream
    else:
        model.fb[i].setlb(0)
        model.fb[i].setub(1000)
        
model.fc = pyo.VarList()
for i in range(0,23):
    model.fc.add()
for i in range(1, len(model.fc)+1):
    if i == 4 or i == 9:
        model.fc[i].setlb(0)
        model.fc[i].setub(1000) #can raise upperbound because of recycle stream
    else:
        model.fc[i].setlb(0)
        model.fc[i].setub(1000)

model.n_trays = pyo.VarList(initialize=8,bounds=(3,22),domain=pyo.Integers) #number of trays/stages variable 3-22
model.feed_tray = pyo.VarList(initialize=4,bounds=(2,21),domain=pyo.Integers) #feed tray/stage variable 2-21
model.ref_rat = pyo.VarList(initialize=1,bounds=(.75,2),domain=pyo.PositiveReals) #reflux ratio variable .75 -2
model.dist_rate = pyo.VarList(initialize=0,bounds=(0,750)) #distillate rate variable 0 - 1000
model.annual_cost = pyo.VarList() #annual cost = capital cost + annual operating cost

for i in range(1,len(model.COLUMNS)+1):
    model.n_trays.add()
    model.feed_tray.add()
    model.ref_rat.add()
    model.dist_rate.add()
    model.annual_cost.add()

#binary variables   
model.y1 = pyo.Var(domain=pyo.Binary)
model.y2 = pyo.Var(domain=pyo.Binary)

#constraints
#NN input variable constraints
def feed_tray_constr1(model,i):
    ineq = model.feed_tray[i] >= 2
    return ineq
def feed_tray_constr2(model,i):
    ineq = 2*model.feed_tray[i] <= model.n_trays[i]
    return ineq

#these dist rate conraints plus the dist rate range 0 - 750 limit the total flow of each stream to 1000
def dist_rate_constr1(model,i):
    ineq = model.dist_rate[i] >= dist_rate_range[0]*(model.fa[inlet_column[i-1]] + model.fb[inlet_column[i-1]] + model.fc[inlet_column[i-1]])
    return ineq
def dist_rate_constr2(model,i):
    ineq = model.dist_rate[i] <= dist_rate_range[1]*(model.fa[inlet_column[i-1]] + model.fb[inlet_column[i-1]] + model.fc[inlet_column[i-1]])
    return ineq

    
#mass flow constraints
#doing feed split constraints seperately because feed is constant so it's a slightly different form than the other split constraints
def feed_splitA(model): #mass balance for first splitter
    ineq = mol_A*feed_rate == sum(model.fa[j] for j in outlet_split[0])
    return ineq
def feed_mfA(model,i):
    ineq = mol_A*(model.fa[outlet_split[0][i-1]] + model.fb[outlet_split[0][i-1]] + model.fc[outlet_split[0][i-1]]) == model.fa[outlet_split[0][i-1]]
    return ineq

def feed_splitB(model): #mass balance for first splitter
    ineq =  mol_B*feed_rate == sum(model.fb[j] for j in outlet_split[0])
    return ineq
def feed_mfB(model,i):
    ineq = mol_B*(model.fa[outlet_split[0][i-1]] + model.fb[outlet_split[0][i-1]] + model.fc[outlet_split[0][i-1]]) == model.fb[outlet_split[0][i-1]]
    return ineq

def feed_splitC(model): #making sure mole fracs are the same across streams 1 2 and 3
    ineq = mol_C*feed_rate == sum(model.fc[j] for j in outlet_split[0])
    return ineq
def feed_mfC(model,i):
    ineq = mol_C*(model.fa[outlet_split[0][i-1]] + model.fb[outlet_split[0][i-1]] + model.fc[outlet_split[0][i-1]]) == model.fc[outlet_split[0][i-1]]
    return ineq

#split constraints are making sure components flow in out streams add up to input component flow, and making sure mole ratios in out streams are same as mole ratios in in stream
def split_flowA(model,i):
    ineq = model.fa[inlet_split[i]]== sum(model.fa[j] for j in outlet_split[i+1])
    return ineq
def split_mfA(model,i,j):
    ineq = (model.fa[outlet_split[i+1][j-1]] +model.fb[outlet_split[i+1][j-1]] +model.fc[outlet_split[i+1][j-1]]) * model.fa[inlet_split[i]] \
    == model.fa[outlet_split[i+1][j-1]] * (model.fa[inlet_split[i]] + model.fb[inlet_split[i]] + model.fc[inlet_split[i]])
    return ineq

def split_flowB(model,i,j):
    ineq = model.fb[inlet_split[i]]== sum(model.fb[j] for j in outlet_split[i+1])
    return ineq
def split_mfB(model,i,j):
    ineq = (model.fa[outlet_split[i+1][j-1]] +model.fb[outlet_split[i+1][j-1]] +model.fc[outlet_split[i+1][j-1]]) * model.fb[inlet_split[i]] \
    == model.fb[outlet_split[i+1][j-1]] * (model.fa[inlet_split[i]] + model.fb[inlet_split[i]] + model.fc[inlet_split[i]])
    return ineq

def split_flowC(model,i,j):
    ineq = model.fc[inlet_split[i]]== sum(model.fc[j] for j in outlet_split[i+1])
    return ineq
def split_mfC(model,i,j):
    ineq = (model.fa[outlet_split[i+1][j-1]] +model.fb[outlet_split[i+1][j-1]] +model.fc[outlet_split[i+1][j-1]]) * model.fc[inlet_split[i]] \
    == model.fc[outlet_split[i+1][j-1]] * (model.fa[inlet_split[i]] + model.fb[inlet_split[i]] + model.fc[inlet_split[i]])
    return ineq

#mixer constraints are making sure that component flow rates of in = out and making sure the amount mixed into the product streams is what we want.
def mix_A_col(model,i):
    ineq = sum(model.fa[j]for j in inlet_col_mix[i]) == model.fa[outlet_col_mix[i]]
    #return (-10,sum(model.f[j]*model.xa[j] for j in inlet_col_mix[i]) - model.f[outlet_col_mix[i]]*model.xa[outlet_col_mix[i]],10)
    return ineq
def mix_B_col(model,i):
    ineq = sum(model.fb[j] for j in inlet_col_mix[i]) == model.fb[outlet_col_mix[i]]
    #return (-10,sum(model.f[j]*model.xb[j] for j in inlet_col_mix[i]) - model.f[outlet_col_mix[i]]*model.xb[outlet_col_mix[i]],10)
    return ineq
def mix_C_col(model,i):
    ineq = sum(model.fc[j]for j in inlet_col_mix[i]) == model.fc[outlet_col_mix[i]]
    #return (-10,sum(model.f[j]*model.xc[j] for j in inlet_col_mix[i]) - model.f[outlet_col_mix[i]]*model.xc[outlet_col_mix[i]],10)
    return ineq
def mix_A_prod1(model):
    #ineq = sum(model.fa[j] for j in inlet_prod_mix[i]) == prod_comp['P1','A'] exact bounds
    return (1*prod_comp['P1','A'],sum(model.fa[j] for j in inlet_prod_mix[0]),prod_comp['P1','A']*1 + 1e-4) #loose bounds
def mix_A_prod2(model):
    #ineq = sum(model.f[j]*model.xa[j] for j in inlet_prod_mix[i]) == prod_comp['P2','A']
    return (1*prod_comp['P2','A'],sum(model.fa[j] for j in inlet_prod_mix[1]),prod_comp['P2','A']*1 + 1e-4)
def mix_B_prod1(model):
    #ineq = sum(model.fb[j] for j in inlet_prod_mix[i]) == prod_comp['P1','B']
    return (1*prod_comp['P1','B'],sum(model.fb[j] for j in inlet_prod_mix[0]),prod_comp['P1','B']*1 + 1e-4)
def mix_B_prod2(model):
    #ineq = sum(model.fb[j] for j in inlet_prod_mix[i]) == prod_comp['P2','B']
    return (1*prod_comp['P2','B'],sum(model.fb[j] for j in inlet_prod_mix[1]),prod_comp['P2','B']*1 + 1e-4)
def mix_C_prod1(model):
    #ineq = sum(model.fc[j] for j in inlet_prod_mix[i]) == prod_comp['P1','C']
    return (1*prod_comp['P1','C'],sum(model.fc[j] for j in inlet_prod_mix[0]),prod_comp['P1','C']*1 + 1e-4)
def mix_C_prod2(model):
    #ineq = sum(model.fc[j] for j in inlet_prod_mix[i]) == prod_comp['P2','C']
    return (1*prod_comp['P2','C'],sum(model.fc[j] for j in inlet_prod_mix[1]),prod_comp['P2','C']*1+ 1e-4)

#column constraints
nn2_ab= keras.models.load_model('DIST keras model comp. 16 node annual cost_halfrangefeedtray.h5')
nn2_bc= keras.models.load_model('DIST keras model comp. 16 node annual cost_halfrangefeedtray.h5')

model.nn_ab = OmltBlock()
model.nn_bc = OmltBlock()

net_relu_ab = keras_reader.load_keras_sequential(nn2_ab,scaler_ab,input_bounds_ab)
net_relu_bc = keras_reader.load_keras_sequential(nn2_bc,scaler_bc,input_bounds_bc)

formulation_ab = ReluPartitionFormulation(net_relu_ab)
formulation_bc = ReluPartitionFormulation(net_relu_bc)

model.nn_ab.build_formulation(formulation_ab)
model.nn_bc.build_formulation(formulation_bc)

@model.Constraint()
#column 1 inputs
def col1_fa(mdl):
    return mdl.fa[4]  == mdl.nn_ab.inputs[0] * mdl.y1
@model.Constraint()
def col1_fb(mdl):
    return mdl.fb[4] == mdl.nn_ab.inputs[1] * mdl.y1
@model.Constraint()
def col1_fc(mdl):
    return  mdl.fc[4] == mdl.nn_ab.inputs[2] * mdl.y1
@model.Constraint()
def col1_ntray(mdl):
    return mdl.nn_ab.inputs[3] ==  mdl.n_trays[1]
@model.Constraint()
def col1_feedtray(mdl):
    return mdl.nn_ab.inputs[4] ==  mdl.feed_tray[1]
@model.Constraint()
def col1_distrate(mdl):
    return mdl.dist_rate[1] == mdl.nn_ab.inputs[5] * mdl.y1
@model.Constraint()
def col1_ref_rat(mdl):
    return mdl.nn_ab.inputs[6] ==  mdl.ref_rat[1]
#column 2 inputs
@model.Constraint()
def col2_fa(mdl):
    return mdl.fa[9] ==  mdl.nn_bc.inputs[0] * mdl.y2
@model.Constraint()
def col2_fb(mdl):
    return mdl.fb[9] == mdl.nn_bc.inputs[1] * mdl.y2
@model.Constraint()
def col2_fc(mdl):
    return mdl.fc[9] == mdl.nn_bc.inputs[2] * mdl.y2
@model.Constraint()
def col2_ntray(mdl):
    return mdl.n_trays[2] == mdl.nn_bc.inputs[3]
@model.Constraint()
def col2_feedtray(mdl):
    return mdl.feed_tray[2] == mdl.nn_bc.inputs[4]
@model.Constraint()
def col2_distrate(mdl):
    return mdl.dist_rate[2] == mdl.nn_bc.inputs[5] * mdl.y2
@model.Constraint()
def col2_ref_rat(mdl):
    return mdl.ref_rat[2] == mdl.nn_bc.inputs[6]
#columns 1 outputs
@model.Constraint()
def col1_da(mdl):
    return mdl.fa[5] == mdl.nn_ab.outputs[0] * mdl.y1
@model.Constraint()
def col1_db(mdl):
    return mdl.fb[5] == mdl.nn_ab.outputs[1] * mdl.y1
@model.Constraint()
def col1_dc(mdl):
    return mdl.fc[5] == mdl.nn_ab.outputs[2] * mdl.y1

@model.Constraint()
def col1_ba(mdl):
    return mdl.fa[6] == (mdl.fa[4] - mdl.fa[5]) * mdl.y1
    
@model.Constraint()
def col1_bb(mdl):
    return mdl.fb[6] == (mdl.fb[4] - mdl.fb[5]) * mdl.y1
@model.Constraint()
def col1_bc(mdl):
    return mdl.fc[6] == (mdl.fc[4] - mdl.fc[5]) * mdl.y1

@model.Constraint()
def col1_annual_cost(mdl):
    return mdl.annual_cost[1] == mdl.nn_ab.outputs[3]  * mdl.y1

#column 2 outputs
@model.Constraint()
def col2_da(mdl):
    return mdl.fa[10] == mdl.nn_bc.outputs[0] *mdl.y2
@model.Constraint()
def col2_db(mdl):
    return mdl.fb[10] == mdl.nn_bc.outputs[1] * mdl.y2
@model.Constraint()
def col2_dc(mdl):
    return mdl.fc[10] == mdl.nn_bc.outputs[2] * mdl.y2
@model.Constraint()
def col2_ba(mdl):
    return mdl.fa[11] == (mdl.fa[9] - mdl.fa[10]) * mdl.y2
@model.Constraint()
def col2_bb(mdl):
    return mdl.fb[11] == (mdl.fb[9] - mdl.fb[10]) * mdl.y2
@model.Constraint()
def col2_bc(mdl):
    return mdl.fc[11] == (mdl.fc[9] - mdl.fc[10]) * mdl.y2
@model.Constraint()
def col2_annual_cost(mdl):
    return mdl.annual_cost[2] == mdl.nn_bc.outputs[3] * mdl.y2


model.feed_tray_constrnt = pyo.Constraint(model.ci,rule=feed_tray_constr1)
model.feed_tray_constrnt2 = pyo.Constraint(model.ci,rule=feed_tray_constr2)

model.dist_rate_constr1 = pyo.Constraint(model.ci,rule=dist_rate_constr1)
model.dist_rate_constr2 = pyo.Constraint(model.ci,rule=dist_rate_constr2)

#model.mole_frac_eq_constr = pyo.Constraint(model.str_i,rule=mole_frac_eq)

#model.feed_splitflow_constr = pyo.Constraint(model.fi,rule=feed_splitflow)
model.feed_splitA_constr = pyo.Constraint(rule=feed_splitA)
model.feed_mfA_constr = pyo.Constraint(model.fi,rule=feed_mfA)
model.feed_splitB_constr = pyo.Constraint(rule=feed_splitB)
model.feed_mfB_constr = pyo.Constraint(model.fi,rule=feed_mfB)
model.feed_splitC_constr = pyo.Constraint(rule=feed_splitC)
model.feed_mfC_constr = pyo.Constraint(model.fi,rule=feed_mfC)

model.splitA_constr = pyo.Constraint(model.si,rule=split_flowA)
model.split_mfA_constr = pyo.Constraint(model.si,model.ci,rule=split_mfA)
model.splitB_constr = pyo.Constraint(model.si,model.ci,rule=split_flowB)
model.split_mfB_constr = pyo.Constraint(model.si,model.ci,rule=split_mfB)
model.splitC_constr = pyo.Constraint(model.si,model.ci,rule=split_flowC)
model.split_mfC_constr = pyo.Constraint(model.si,model.ci,rule=split_mfC)

model.mix_A_col_constr = pyo.Constraint(model.mci,rule=mix_A_col)
model.mix_B_col_constr = pyo.Constraint(model.mci,rule=mix_B_col)
model.mix_C_col_constr = pyo.Constraint(model.mci,rule=mix_C_col)


model.mix_A_prod1_constr = pyo.Constraint(rule=mix_A_prod1)
model.mix_A_prod2_constr = pyo.Constraint(rule=mix_A_prod2)
model.mix_B_prod1_constr = pyo.Constraint(rule=mix_B_prod1)
model.mix_B_prod2_constr = pyo.Constraint(rule=mix_B_prod2)
model.mix_C_prod1_constr = pyo.Constraint(rule=mix_C_prod1)
model.mix_C_prod2_constr = pyo.Constraint(rule=mix_C_prod2)

#Objective function
model.obj = pyo.Objective(expr=(model.annual_cost[1] + model.annual_cost[2]),sense=pyo.minimize)

In [7]:
#model.pprint()
now = datetime.now()
date_time_str = now.strftime("%Y-%m-%d %H.%M")
solver = pyo.SolverFactory('gurobi')
filename = "unimportant" + date_time_str + ".txt"
solver.options["LogFile"] = filename  # Specify the log file name
status = solver.solve(model, tee=True,  options={'TimeLimit': 30})

Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-29
Read LP format model from file C:\Users\RazzyZac\AppData\Local\Temp\tmp6e3q93zh.pyomo.lp
Reading time = 0.02 seconds
x1: 527 rows, 319 columns, 2183 nonzeros
Set parameter LogFile to value "unimportant2024-07-03 14.11.txt"
Set parameter TimeLimit to value 30
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 527 rows, 319 columns and 2183 nonzeros
Model fingerprint: 0x08241232
Model has 64 quadratic constraints
Variable types: 281 continuous, 38 integer (34 binary)
Coefficient statistics:
  Matrix range     [2e-03, 8e+01]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e-03, 1e+03]
  RHS range        [1e-02, 

ApplicationError: Solver (gurobi) did not exit normally